In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
import pandas as pd
import numpy as np

df_demograficos = pd.read_csv("/content/drive/MyDrive/Data/BDIPMamaV11-INTELIDemograficosTt_DATA_LABELS_2023-01-24_1922 - BDIPMamaV11-INTELIDemograficosTt_DATA_LABELS_2023-01-24_1922genesis.csv")
df_histopatologia = pd.read_csv("/content/drive/MyDrive/Data/BDIPMamaV11-INTELIHistopatologia_DATA_LABELS_2023-01-24_1924.csv")
df_registro_tumo = pd.read_csv("/content/drive/MyDrive/Data/BDIPMamaV11-INTELIRegistroDeTumo_DATA_LABELS_2023-01-24_1924.csv")
df_pesoEaltura = pd.read_csv("/content/drive/MyDrive/Data/BDIPMamaV11-INTELIPesoEAltura_DATA_LABELS_2023-01-24_1926.csv")

In [25]:
df_demograficos = df_demograficos.drop(['Repeat Instrument', 
                                        'Repeat Instance',
                                        'UF de nascimento do paciente',
                                        'UF de residência do paciente',
                                        'Quantas vezes ficou gráfida?',
                                        'Número de partos',
                                        'Hormonioterapia ',
                                        'Amamentou na primeira gestação?',
                                        'Faz uso de métodos contraceptivo?',
                                        'Esquema de hormonioterapia',
                                        'Esquema de hormonioterapia',
                                        'Data do início Hormonioterapia adjuvante',
                                        'Data de início da Radioterapia',
                                        'Radioterapia',
                                        'Consumo de tabaco',
                                        'Abortou'], axis=1)


In [26]:
df_demograficos = df_demograficos.rename(columns={
                        "Idade do paciente ao primeiro diagnóstico": "idade_diagnostico", 
                        "Raça declarada (Biobanco)": "raca",
                        "Última informação do paciente": "ultima_informacao_paciente",
                        "Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]": "tempo_seguimento",
                        "Já ficou grávida?": "ja_gravida",
                        "Idade na primeira gestação": "gestacao_idade",
                        "Consumo de álcool": "consumo_alcool",
                        "Possui histórico familiar de câncer?": "historico_cancer",
                        "Grau de parentesco de familiar com cancer? (choice=primeiro (pais, irmãos, filhos))": "grau_parentesco_primeiro",
                        "Grau de parentesco de familiar com cancer? (choice=segundo (avós, tios e netos))": "grau_parentesco_segundo",
                        "Grau de parentesco de familiar com cancer? (choice=terceiro (bisavós, tio avós, primos, sobrinhos))": "grau_parentesco_terceiro",
                        "Regime de Tratamento": "tratamento",
                        "Tipo de terapia anti-HER2 neoadjuvante": "anti_her2_neoadjuvante"
                        })

df_demograficos = df_demograficos.drop(df_demograficos[df_demograficos['tratamento'] == 'Paliativo'].index)
df_demograficos = df_demograficos.drop(df_demograficos[df_demograficos['tratamento'] == 'Não fez quimioterapia'].index)

lista_replace = [["Escolaridade",
                  "IGNORADA", "ANALFABETO", "ENS. FUNDAMENTAL INCOMPLETO", "ENS. FUNDAMENTAL COMPLETO", "ENSINO MÉDIO", "SUPERIOR"],
                 ["Sexo", 
                  "Feminino", "Masculino"], 
                 ["raca",
                  "Negro", "Pardo", "Asiático", "Branco", "Outro"],
                 ["ja_gravida",
                  "Não", "Sim"],
                 ["consumo_alcool",
                  "Bebe atualmente","Bebia no passado","não-informado","Nunca bebeu"],
                 ["historico_cancer",
                  "Não","Sim"],
                 ["ultima_informacao_paciente",
                  "Vivo, SOE", "Vivo, com câncer", "Óbito por outras causas, SOE", "Obito por câncer"],
                 ["grau_parentesco_primeiro",
                  "Unchecked", "Checked"],
                 ["grau_parentesco_segundo",
                 "Unchecked", "Checked"],
                 ["grau_parentesco_terceiro",
                  "Unchecked", "Checked"],
                 ["tratamento",
                  "Terapia Adjuvante", "Terapia Neoadjuvante"],
                 ["anti_her2_neoadjuvante",
                  "Trastuzumabe","Trastuzumabe + Pertuzumabe"]
                 ]

for i in range(0, len(lista_replace)):
  for j in range(1, len(lista_replace[i])):
    df_demograficos[lista_replace[i][0]].replace(lista_replace[i][j], j-1, inplace=True)

In [27]:
np.random.seed(73)

df_demograficos = df_demograficos.drop(df_demograficos[df_demograficos['tratamento'] == 'Paliativo'].index)
df_demograficos = df_demograficos.drop(df_demograficos[df_demograficos['tratamento'] == 'Não fez quimioterapia'].index)

lista = [
    ["Sexo", 0, 1], 
    ["Escolaridade", 0, 5], ["idade_diagnostico"] , ["raca", 0, 4],
    ["ja_gravida", 0 , 1], ["consumo_alcool", 0 , 3],["gestacao_idade"],
    ["historico_cancer"],["grau_parentesco_primeiro", 0, 1],["grau_parentesco_segundo", 0, 1], 
    ["tempo_seguimento"], ["grau_parentesco_terceiro", 0, 1], ["anti_her2_neoadjuvante",0, 1],
    ['tratamento',0, 1]
         ]

lista_normal = []

for i in range(0, len(lista)): 
  lista_normal.append(np.random.normal(df_demograficos[lista[i][0]].mean(skipna=True), 
                                       df_demograficos[lista[i][0]].std(skipna=True), 
                                       df_demograficos[f'{lista[i][0]}'].isnull().sum()))

  for j in range(0, len(lista_normal[i])):
    lista_normal[i][j] = round(lista_normal[i][j])

    if len(lista[i]) >= 2 : 
      if lista_normal[i][j] > lista[i][2]: 
        lista_normal[i][j] = lista[i][2]
      if lista_normal[i][j] < lista[i][1]: 
        lista_normal[i][j] = lista[i][1]

for i in range(0, len(lista)):
  df_demograficos[f'{lista[i][0]}'] = df_demograficos[f'{lista[i][0]}'].fillna(pd.Series(lista_normal[i], index=df_demograficos[f'{lista[i][0]}'].index[df_demograficos[f'{lista[i][0]}'].isnull()]))

df_demograficos

,Record ID,Escolaridade,idade_diagnostico,Sexo,raca,Data da última informação sobre o paciente,ultima_informacao_paciente,tempo_seguimento,ja_gravida,gestacao_idade,...,Atividade Física,consumo_alcool,historico_cancer,grau_parentesco_primeiro,grau_parentesco_segundo,grau_parentesco_terceiro,tratamento,Data da cirurgia,anti_her2_neoadjuvante,Data de início do tratamento quimioterapia
0,302,2.0,51.0,0.0,0.0,2014-04-26,3.0,2225.0,1.0,22.0,...,NaN,2.0,1.0,0,0,0,0.0,NaN,0.0,NaN
1,710,4.0,58.0,0.0,2.0,2016-11-17,0.0,3294.0,1.0,29.0,...,NaN,2.0,0.0,0,0,0,0.0,2009-09-04,0.0,2014-08-24
2,752,2.0,56.0,0.0,2.0,2019-05-02,0.0,4153.0,1.0,12.0,...,NaN,3.0,1.0,0,0,0,1.0,NaN,0.0,NaN
3,1367,2.0,63.0,0.0,1.0,2011-09-29,3.0,1331.0,1.0,26.0,...,NaN,3.0,0.0,0,0,0,1.0,2011-07-05,0.0,NaN
4,1589,3.0,42.0,0.0,1.0,2017-05-24,0.0,3290.0,1.0,26.0,...,NaN,1.0,2.0,0,0,0,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4267,82123,0.0,57.0,0.0,2.0,2021-10-25,0.0,380.0,1.0,30.0,...,NaN,3.0,0.0,0,0,0,1.0,2021-07-04,0.0,2020-12-14
4268,82124,1.0,41.0,0.0,0.0,2021-01-21,3.0,138.0,1.0,27.0,...,NaN,2.0,0.0,0,0,0,1.0,NaN,0.0,NaN
4269,82131,4.0,59.0,0.0,3.0,2022-06-10,3.0,900.0,1.0,26.0,...,NaN,2.0,1.0,0,0,0,0.0,2020-12-23,0.0,NaN
4270,82205,1.0,29.0,0.0,2.0,2022-04-29,3.0,538.0,1.0,16.0,...,NaN,2.0,1.0,0,0,0,1.0,NaN,0.0,NaN


In [ ]:
df_histopatologia = pd.read_csv("/content/drive/MyDrive/Data/BDIPMamaV11-INTELIHistopatologia_DATA_LABELS_2023-01-24_1924.csv")

df_histopatologia = df_histopatologia.drop(['Repeat Instrument', 
                                        'Repeat Instance',
                                        'Receptor de progesterona (quantificação %)',
                                        'Receptorde Estrogênio (quantificação %)',
                                        'Indice H (Receptorde progesterona)',
                                        'Ki67 (%)',
                                        ], axis=1)

df_histopatologia = df_histopatologia.rename(columns={"Record ID": "record_id"})

lista_RecorId = df_histopatologia['record_id'].unique()

df_novo_histopatologia = df_histopatologia 
df_novo = pd.DataFrame(columns= df_histopatologia.columns)

for i in lista_RecorId:
    df_query = df_histopatologia.query(f'record_id == {i}')
    idx_linha_max = df_query.apply(lambda x: x.count(), axis=1).idxmax()
    linha_max = df_query.loc[idx_linha_max]
    df_novo = df_novo.append(linha_max, ignore_index=True)

df_histopatologia = df_novo

df_histopatologia

,record_id,Diagnostico primario (tipo histológico),Grau histológico,Subtipo tumoral,Receptor de estrogênio,Receptor de progesterona,Ki67 (>14%),HER2 por IHC,HER2 por FISH
0,302,NaN,NaN,5.0,positivo,positivo,NaN,+++ (positivo),NaN
1,710,NaN,NaN,2.0,positivo,negativo,positivo,0 (negativo),NaN
2,752,NaN,NaN,3.0,NaN,NaN,NaN,+++ (positivo),NaN
3,1367,NaN,NaN,2.0,positivo,positivo,positivo,0 (negativo),NaN
4,1589,NÃO-ESPECIAL - Carcinoma de mama ductal invasi...,NaN,5.0,positivo,positivo,NaN,+++ (positivo),NaN
...,...,...,...,...,...,...,...,...,...
4267,82123,NaN,NaN,3.0,NaN,NaN,positivo,+++ (positivo),NaN
4268,82124,NaN,NaN,3.0,NaN,NaN,NaN,+++ (positivo),NaN
4269,82131,NaN,NaN,3.0,NaN,NaN,positivo,+++ (positivo),NaN
4270,82205,NaN,NaN,4.0,negativo,negativo,positivo,0 (negativo),NaN


In [63]:
df_histopatologia = df_histopatologia.rename(columns={
                        "Grau histológico": "grau_histologico", 
                        "Subtipo tumoral": "subtipo_tumoral",
                        "Receptor de estrogênio": "receptor_estrogenio",
                        "Receptor de progesterona": "receptor_progesterona",
                        "Ki67 (>14%)": "ki67",
                        "HER2 por IHC": "her2_ihc",
                        "Diagnostico primario (tipo histológico)": "primeiro_diagnostico",
                        "HER2 por FISH": "her2_fish"
                        })

In [64]:
lista_replace = [["primeiro_diagnostico",
                  "NÃO-ESPECIAL - Carcinoma de mama ductal invasivo (CDI)/SOE", "Carcinoma de mama lobular invasivo",
                    "outros", "Carcinoma de mama metaplasico", "Carcinoma de mama mucinoso", "Carcinoma de mama papilifero",
                    "Carcinoma de mama medular", "Carcinoma de mama misto (ductal e lobular) invasivo", 
                    "Carcinoma de mama cistico adenoide", "Carcinoma mamário invasivo multifocal",
                    "Carcinoma de mama misto (ductal e micropapilífero) invasivo", "CARCINOMA MAMÁRIO INVASIVO DO TIPO APÓCRINO",
                    "Carcinoma de mama lobular in situ", "Carcinoma lobular pleomórfico", "Adenomioepitelioma maligno",
                    "Carcinoma de mama tubular", "Tumor PHYLLODES maligno", "Carcinoma de mama micropapilar"],
                 ["receptor_estrogenio", 
                  "positivo", "negativo", "não realizado"], 
                 ["receptor_progesterona",
                  "positivo", "negativo", "inconclusivo", "não realizado"],
                 ["ki67",
                  "positivo","negativo","inconclusivo","não realizado"],
                 ["her2_ihc",
                  " +++ (positivo)"," ++ (duvidoso)"," + (negativo)","indeterminado", "0 (negativo)"],
                 ["her2_fish",
                  "não realizado", "amplificado", "sem amplificação", "duvidoso", "reação não funcionou"]
                 ]

for i in range(0, len(lista_replace)):
  for j in range(1, len(lista_replace[i])):
    df_histopatologia[lista_replace[i][0]].replace(lista_replace[i][j], j-1, inplace=True)

array([nan,  0.,  4.,  1.,  3.,  2.])

In [65]:
np.random.seed(73)

lista = [
    ["primeiro_diagnostico", 0, 17], 
    ["grau_histologico"], ["subtipo_tumoral", 0 , 5] , ["receptor_estrogenio", 0, 2],
    ["receptor_progesterona", 0 , 3], ["ki67", 0 , 3],["her2_ihc", 0 , 4],
    ["her2_fish", 0, 4]
         ]

lista_normal = []

for i in range(0, len(lista)): 
  lista_normal.append(np.random.normal(df_histopatologia[lista[i][0]].mean(skipna=True), 
                                       df_histopatologia[lista[i][0]].std(skipna=True), 
                                       df_histopatologia[f'{lista[i][0]}'].isnull().sum()))

  for j in range(0, len(lista_normal[i])):
    lista_normal[i][j] = round(lista_normal[i][j])

    if len(lista[i]) >= 2 : 
      if lista_normal[i][j] > lista[i][2]: 
        lista_normal[i][j] = lista[i][2]
      if lista_normal[i][j] < lista[i][1]: 
        lista_normal[i][j] = lista[i][1]

for i in range(0, len(lista)):
  df_histopatologia[f'{lista[i][0]}'] = df_histopatologia[f'{lista[i][0]}'].fillna(pd.Series(lista_normal[i], index=df_histopatologia[f'{lista[i][0]}'].index[df_histopatologia[f'{lista[i][0]}'].isnull()]))

df_histopatologia

,Record ID,Repeat Instrument,Repeat Instance,primeiro_diagnostico,grau_histologico,subtipo_tumoral,receptor_estrogenio,receptor_progesterona,ki67,Receptor de progesterona (quantificação %),Receptorde Estrogênio (quantificação %),Indice H (Receptorde progesterona),her2_ihc,her2_fish,Ki67 (%)
0,302,NaN,NaN,2.0,2.0,5.0,0.0,1.0,0.0,NaN,NaN,NaN,2.0,0.0,NaN
1,302,Dados Histopatologicos Mama,1.0,4.0,3.0,5.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN
2,710,NaN,NaN,5.0,3.0,3.0,1.0,0.0,0.0,NaN,NaN,NaN,4.0,0.0,NaN
3,710,Dados Histopatologicos Mama,1.0,1.0,0.0,2.0,0.0,1.0,0.0,NaN,NaN,NaN,4.0,1.0,20.0
4,752,NaN,NaN,0.0,2.0,5.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9061,82131,Dados Histopatologicos Mama,1.0,0.0,1.0,3.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,50.0
9062,82205,NaN,NaN,0.0,3.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN,2.0,2.0,NaN
9063,82205,Dados Histopatologicos Mama,1.0,2.0,3.0,4.0,1.0,1.0,0.0,NaN,NaN,NaN,4.0,1.0,90.0
9064,82240,NaN,NaN,1.0,3.0,3.0,0.0,1.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN


In [121]:
df_registro_tumo = pd.read_csv("/content/drive/MyDrive/Data/BDIPMamaV11-INTELIRegistroDeTumo_DATA_LABELS_2023-01-24_1924.csv")

df_registro_tumo = df_registro_tumo.drop(['Repeat Instrument', 
                                        'Repeat Instance'
                                        ], axis=1)

In [122]:
df_registro_tumo = df_registro_tumo.rename(columns={
                        "Record ID" : "record_id",
                        "Código da Topografia (CID-O)": "cid_o", 
                        "Código da Morfologia de acordo com o CID-O": "morfologia_cid_o",
                        "Estadio Clínico  ": "estadio_clinico",
                        "Grupo de Estadio Clínico": "grupo_estadio_clinico",
                        "Metastase ao DIAGNOSTICO - CID-O #4 ": "metastase_cido4",
                        "Classificação TNM Clínico - M": "classificacao_tnm_m",
                        "Classificação TNM Clínico - N": "classificacao_tnm_n",
                        "Classificação TNM Patológico - N": "classificacao_tnm_n_patologico",
                        "Classificação TNM Patológico - T": "classificacao_tnm_t_patologico",
                        "Com recidiva à distância": "recidiva_distancia",
                        "Com recidiva regional": "recidiva_regional",
                        "Com recidiva local": "recidiva_local",
                        "Combinação dos Tratamentos Realizados no Hospital": "combinacao_tratamentos",
                        "Data de Recidiva": "data_recidiva",
                        "Lateralidade do tumor": "lateralidade_tumor",
                        "Data do tratamento": "data_tratamento",
                        "Tempo desde o diagnóstico até a primeira recidiva  ": "tempo_diagnostico",
                        "Descrição da Morfologia de acordo com o CID-O  (CID-O - 3ª edição)": "descricao_morfologia",
                        "Descrição da Topografia ": "descricao_topografia"
                        })

lista_replace = [["grupo_estadio_clinico",
                  "0", "I", "II", "III", "III", "IV", "Y: Na", "X - nao foi possivel determinar"],
                 ["lateralidade_tumor", 
                  "Esquerda", "Direita", "não se aplica", "Bilateral"], 
                 ["combinacao_tratamentos",
                  "Cirurgia + Radio + Quimio + Hormonio", "Cirurgia + Quimioterapia",
                    "Outras combinações", "Cirurgia + Radio + Quimio", "Cirurgia + Radioterapia",
                    "Quimioterapia", "Radioterapia + Quimioterapia", "Nenhum tratamento", "Radioterapia",
                    "Cirurgia"],
                 ["classificacao_tnm_m",
                  "0", "1", "Y: Na", "X - nao foi possivel determinar", "1B", "1A"],
                 ["classificacao_tnm_n",
                  "0","1","2","3","1A","2A","3A","1B","2B","3B","3C","X - nao foi possivel determinar",
                  "Y: Na"],
                 ["classificacao_tnm_t_patologico",
                  "2","1C","3","1","1B","4B","IV","IS","4D","1MIC","Y: Na","2C","1A","4A","4C"],
                 ["classificacao_tnm_n_patologico",
                  "0", "1", "2", "3", "2A", "3A", "3B", "3C", "Y: Na", "X - nao foi possivel determinar","9", "10", "11"],
                 ["recidiva_distancia",
                  "Não", "Sim"],
                 ["recidiva_regional",
                 "Não", "Sim"],
                 ["recidiva_local",
                  "Não", "Sim"]
                 ]
                 
for i in range(0, len(lista_replace)):
  for j in range(1, len(lista_replace[i])):
    df_registro_tumo[lista_replace[i][0]].replace(lista_replace[i][j], j-1, inplace=True)

In [123]:
lista_RecorId = df_registro_tumo['record_id'].unique()

df_novo_histopatologia = df_registro_tumo 
df_novo = pd.DataFrame(columns= df_registro_tumo.columns)

for i in lista_RecorId:
    df_query = df_registro_tumo.query(f'record_id == {i}')
    idx_linha_max = df_query.apply(lambda x: x.count(), axis=1).idxmax()
    linha_max = df_query.loc[idx_linha_max]
    df_novo = df_novo.append(linha_max, ignore_index=True)

df_registro_tumo = df_novo


In [130]:
np.random.seed(73)

lista = [
    ["grupo_estadio_clinico", 0, 6],
    ["combinacao_tratamentos",0, 22], ["tempo_diagnostico", 0 , 22] , ["classificacao_tnm_m", 0, 5],
    ["classificacao_tnm_n", 0 , 3], ["morfologia_cid_o", 0 , 22],["classificacao_tnm_n_patologico", 0 , 22],
    ["classificacao_tnm_t_patologico", 0, 22], ["grupo_estadio_clinico"], ["recidiva_distancia", 0 , 1], ["recidiva_regional"], ["recidiva_local", 0, 1]
         ]

lista_normal = []

for i in range(0, len(lista)):
  lista_normal.append(np.random.normal(df_registro_tumo[lista[i][0]].mean(skipna=True),
                                      df_registro_tumo[lista[i][0]].std(skipna=True),
                                      df_registro_tumo[f'{lista[i][0]}'].isnull().sum()))
  for j in range(0, len(lista_normal[i])):
    lista_normal[i][j] = round(lista_normal[i][j])

    if len(lista[i]) >= 2 : 
      if lista_normal[i][j] > lista[i][2]: 
        lista_normal[i][j] = lista[i][2]
      if lista_normal[i][j] < lista[i][1]: 
        lista_normal[i][j] = lista[i][1]

null_cid_o = df_registro_tumo.cid_o.isnull().sum()
lista_normal_cid_o = []

null_descricao_topografia = df_registro_tumo.descricao_topografia.isnull().sum()
lista_normal_descricao_topografia = []

null_descricao_morfologia = df_registro_tumo.descricao_morfologia.isnull().sum()
lista_normal_descricao_morfologia = []

for i in range(0, null_cid_o):
  lista_normal_cid_o.append("C509")

for i in range(0, null_descricao_morfologia): 
  lista_normal_descricao_morfologia.append("CARCINOMA DUCTAL INFILTRANTE SOE")

for i in range(0, null_descricao_topografia): 
  lista_normal_descricao_topografia.append("MAMA SOE (EXCLUI PELE DA MAMA C44.5)")

for i in range(0, len(lista)):
  df_registro_tumo[f'{lista[i][0]}'] = df_registro_tumo[f'{lista[i][0]}'].fillna(pd.Series(lista_normal[i], index=df_registro_tumo[f'{lista[i][0]}'].index[df_registro_tumo[f'{lista[i][0]}'].isnull()]))

df_registro_tumo


,record_id,Data da primeira consulta institucional [dt_pci],Data do diagnóstico,cid_o,morfologia_cid_o,estadio_clinico,grupo_estadio_clinico,Classificação TNM Clínico - T,classificacao_tnm_n,classificacao_tnm_m,...,Local de Recidiva a distancia/ metastase #2 - CID-O - Topografia,Local de Recidiva a distancia/ metastase #3 - CID-O - Topografia,Local de Recidiva a distancia/ metastase #4 - CID-O - Topografia,descricao_morfologia,descricao_topografia,classificacao_tnm_n_patologico,classificacao_tnm_t_patologico,recidiva_distancia,recidiva_regional,recidiva_local
0,302,2008-03-22,2008-03-23,C504,85003.0,IIA,2.0,2,0.0,0.0,...,NaN,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA QUADRANTE SUPERIOR EXTERNO DA,0.0,0.0,0.0,0.0,0.0
1,710,2006-11-11,2007-11-11,C508,85003.0,IIIA,3.0,3,1.0,0.0,...,NaN,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA LESAO SOBREPOSTA DA,1.0,7.0,0.0,1.0,1.0
2,752,2007-09-25,2007-12-18,C509,84803.0,IIA,2.0,2,0.0,0.0,...,NaN,NaN,NaN,ADENOCARCINOMA MUCINOSO,MAMA SOE (EXCLUI PELE DA MAMA C44.5),9.0,0.0,0.0,0.0,0.0
3,1367,2008-02-03,2008-02-06,C505,85003.0,IIA,2.0,1,1.0,0.0,...,C50 - Mama,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA QUADRANTE INFERIOR EXTERNO DA,1.0,12.0,0.0,1.0,0.0
4,1589,2008-05-15,2008-05-21,C508,85003.0,IIB,2.0,2,1.0,0.0,...,NaN,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA LESAO SOBREPOSTA DA,1.0,4.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4267,82123,2020-12-04,2020-10-10,C504,85003.0,IIB,3.0,3,0.0,0.0,...,NaN,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA QUADRANTE SUPERIOR EXTERNO DA,1.0,2.0,0.0,0.0,0.0
4268,82124,2020-06-20,2020-09-05,C509,85203.0,IV,2.0,4D,1.0,1.0,...,NaN,NaN,NaN,CARCINOMA LOBULAR SOE,MAMA SOE (EXCLUI PELE DA MAMA C44.5),1.0,4.0,0.0,0.0,0.0
4269,82131,2020-11-01,2019-12-23,C502,85203.0,IIIA,4.0,3,1.0,0.0,...,NaN,NaN,NaN,CARCINOMA LOBULAR SOE,MAMA QUADRANTE SUPERIOR INTERNO DA,0.0,6.0,0.0,0.0,0.0
4270,82205,2021-02-28,2020-11-07,C504,85003.0,IV,2.0,4D,1.0,1.0,...,NaN,NaN,NaN,CARCINOMA DUCTAL INFILTRANTE SOE,MAMA QUADRANTE SUPERIOR EXTERNO DA,3.0,0.0,0.0,0.0,1.0
